<a href="https://colab.research.google.com/github/lazybuttrying/AI_Project/blob/main/leaf_segmentation/MaskRCNN/plant_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### 모든 사진의 이름이 겹치지 않도록 조정

In [92]:
cd /gdrive/MyDrive/AI/maskrcnn/plant_images

/gdrive/MyDrive/AI/maskrcnn/plant_images


In [4]:
ls

Ara2012/  Ara2013-Canon/  Ara2013-RPi/


In [6]:
import os

len(os.listdir("./Ara2012"))//3, len(os.listdir("./Ara2013-Canon"))//3, len(os.listdir("./Ara2013-RPi"))//3,
#Ara2013-RPi has No Labled Image :(

(16, 20, 9)

In [7]:
# import os

# for folder in ['Ara2012', 'Ara2013-Canon', 'Ara2013-RPi']:
#   for fname in os.listdir("./"+folder):
#     half = fname.find("_")
#     old = ("./"+folder+"/"+fname)
#     new = ("./"+folder+"/"+fname[:half]+ "~" + folder + fname[half:])
#     os.rename(old, new)

# Image to COCO annotation

In [8]:
cd /gdrive/MyDrive/AI/maskrcnn/

/gdrive/MyDrive/AI/maskrcnn


In [ ]:
!git clone https://github.com/chrise96/image-to-coco-json-converter.git

Cloning into 'image-to-coco-json-converter'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 58 (delta 12), reused 51 (delta 10), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [9]:
cd image-to-coco-json-converter

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter


In [10]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.2 MB 22.5 MB/s 
     |████████████████████████████████| 1.0 MB 43.8 MB/s 
     |████████████████████████████████| 12.5 MB 46.7 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: Shapely
    Found existing installation: Shapely 1.8.0
    Uninstalling Shapely-1.8.0:
      Successfully uninstalled Shapely-1.8.0
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

### label 이미지 마련하기

In [69]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset


In [70]:
!mkdir plant_img
!mkdir plant_img/train_mask
!mkdir plant_img/val_mask

In [71]:
cd /gdrive/MyDrive/AI/maskrcnn/plant_images

/gdrive/MyDrive/AI/maskrcnn/plant_images


In [72]:
# train 24, val 6, (test 9)

!cp Ara2012/*_fg.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img/val_mask
!cp Ara2013-Canon/*_fg.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img/train_mask

In [73]:
import shutil
path ='/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img/'
for fname in sorted(os.listdir(path+'val_mask/'))[:10]:
  #print(fname)
  shutil.move(path+'val_mask/'+fname, path+'train_mask/'+fname)

In [75]:
os.listdir((path+'train_mask/'))[:15] #2012에서 10개 추가

['ara2012_tray01_fg.png',
 'ara2012_tray02_fg.png',
 'ara2012_tray03_fg.png',
 'ara2012_tray04_fg.png',
 'ara2012_tray05_fg.png',
 'ara2012_tray06_fg.png',
 'ara2012_tray07_fg.png',
 'ara2012_tray08_fg.png',
 'ara2012_tray09_fg.png',
 'ara2012_tray10_fg.png',
 'ara2013_tray01_fg.png',
 'ara2013_tray02_fg.png',
 'ara2013_tray03_fg.png',
 'ara2013_tray04_fg.png',
 'ara2013_tray05_fg.png']

In [77]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img


In [80]:
# label 이름으로 저장되면 나중에 사진 이름과 매치되지 않아서 작동 안 됨
# json에 기록될 파일 이름이 실제 파일과 같아지도록 수정

import os

for path in ['./train_mask', './val_mask']:
  for fname in os.listdir(path):
    half = fname.find("f")
    old = path+"/"+fname
    new = path+"/"+fname[:half]+"rgb.png"
    #print(new)
    os.rename(old, new)

### 변환 코드

In [81]:
cd ../../

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter


In [82]:
import glob

from src.create_annotations import *
def get_json(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    cnt = 1
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]
        
            if pixel != (0, 0, 0): # 배경은 제외
              # Check to see if we have created a sub-mask...
              pixel_str = str(pixel)
              sub_mask = sub_masks.get(pixel_str)
              if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                  # Note: we add 1 pixel of padding in each direction
                  # because the contours module doesn"t handle cases
                  # where pixels bleed to the edge of the image
                  sub_masks[pixel_str] = cnt
                  cnt+=1

              # Set the pixel value to 1 (default is 0), accounting for padding
              #sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [83]:
def create_sub_masks(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            if pixel != (0, 0, 0):
              # Check to see if we have created a sub-mask...
              pixel_str = str(pixel)
              sub_mask = sub_masks.get(pixel_str)
              if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                  # Note: we add 1 pixel of padding in each direction
                  # because the contours module doesn"t handle cases
                  # where pixels bleed to the edge of the image
                  sub_masks[pixel_str] = Image.new("1", (width+2, height+2))

              # Set the pixel value to 1 (default is 0), accounting for padding
              sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


In [ ]:


# # Label ids of the dataset
# category_ids = {
#     "outlier": 0,
#     "window": 1,
#     "wall": 2,
#     "balcony": 3,
#     "door": 4,
#     "roof": 5,
#     "sky": 6,
#     "shop": 7,
#     "chimney": 8
# }

# # Define which colors match which categories in the images
# category_colors = {
#     "(0, 0, 0)": 0, # Outlier
#     "(255, 0, 0)": 1, # Window
#     "(255, 255, 0)": 2, # Wall
#     "(128, 0, 255)": 3, # Balcony
#     "(255, 128, 0)": 4, # Door
#     "(0, 0, 255)": 5, # Roof
#     "(128, 255, 255)": 6, # Sky
#     "(0, 255, 0)": 7, # Shop
#     "(128, 128, 128)": 8 # Chimney
# }

# # Define the ids that are a multiplolygon. In our case: wall, roof and sky
# multipolygon_ids = [2, 5, 6]



In [84]:
import gc
# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".png"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)
        del image

        ###########category_colors = get_json(mask_image_open, w, h)
        # print(category_colors)

        sub_masks = create_sub_masks(mask_image_open, w, h)

        del mask_image_open
        gc.collect()

        for color, sub_mask in sub_masks.items():
            # print(sub_masks,category_colors.keys(), color)
          
            ###########category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # # Check if we have classes that are a multipolygon
            # if category_id in multipolygon_ids:
            #     # Combine the polygons to calculate the bounding box and area
            #     multi_poly = MultiPolygon(polygons)
                                
            #     annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

            #     annotations.append(annotation)
            #     annotation_id += 1
            # else:
            
            for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, i, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id


In [ ]:
get_coco_json_format()

{'annotations': [{}],
 'categories': [{}],
 'images': [{}],
 'info': {},
 'licenses': []}

In [ ]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter


In [ ]:
pwd

'/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter'

In [85]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    
    for keyword in ["train", "val"]:
        mask_path = "dataset/plant_img/{}_mask/".format(keyword)
        
        # Create category section
        # coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)
        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

Created 2597 annotations for images in folder: dataset/plant_img/train_mask/
Created 648 annotations for images in folder: dataset/plant_img/val_mask/


# COCO to VGG
https://annotate.officialstatistics.org/

coco 로 붙러와서 json으로 저장

via_region_data.json라는 이름으로 

google drive의 dataset/leaf_img/train_mask/ 와 dataset/leaf_img/val_mask/

각각에 올려두기. 



### VGG annotation 손보기

In [ ]:
# import json
# import pprint


# with open('/content/via_region_data (3).json', 'r') as f:

#     json_data = json.load(f)

# # print(json.dumps(json_data, indent="\t") )

# for k,v in json_data.items():
#   tmp = {}
#   for i in range(len(v["regions"])):
#     #print(v["regions"])
#     v["regions"][i]["region_attributes"] = {}
#     tmp[i] = v["regions"][i]
#   #pprint.pprint(tmp)
#   v["regions"] = tmp

# with open('/content/via_region_data.json', 'w', encoding='utf-8') as make_file:

#     json.dump(json_data, make_file, indent="\t")

# 마무리된 dataset 옮기기

In [86]:
!rm -d -r /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train
!rm -d -r /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/val
!mkdir /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train
!mkdir /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/val

rm: cannot remove '/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train': No such file or directory
rm: cannot remove '/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/val': No such file or directory


In [88]:
pwd # 현재위치확인

'/gdrive/MyDrive/AI/maskrcnn/plant_images'

In [87]:
cd /gdrive/MyDrive/AI/maskrcnn/plant_images/

/gdrive/MyDrive/AI/maskrcnn/plant_images


In [90]:
# 원본 이미지 이동
!cp Ara2013-Canon/*_rgb.png /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train/

In [95]:
import shutil
#path ='/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/plant_img/'
i = 0
for fname in sorted(os.listdir(('./Ara2012/'))):
  #print(fname)
  if "rgb" in fname:
    i+=1
    if i<=10:
      #print(fname)
      shutil.copyfile('./Ara2012/'+fname, '/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train/'+fname)
    else:
      shutil.copyfile('./Ara2012/'+fname, '/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/val/'+fname)

In [96]:
os.listdir('/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/train/')

['ara2013_tray01_rgb.png',
 'ara2013_tray02_rgb.png',
 'ara2013_tray03_rgb.png',
 'ara2013_tray04_rgb.png',
 'ara2013_tray05_rgb.png',
 'ara2013_tray06_rgb.png',
 'ara2013_tray07_rgb.png',
 'ara2013_tray08_rgb.png',
 'ara2013_tray09_rgb.png',
 'ara2013_tray10_rgb.png',
 'ara2013_tray11_rgb.png',
 'ara2013_tray12_rgb.png',
 'ara2013_tray13_rgb.png',
 'ara2013_tray14_rgb.png',
 'ara2013_tray15_rgb.png',
 'ara2013_tray16_rgb.png',
 'ara2013_tray17_rgb.png',
 'ara2013_tray18_rgb.png',
 'ara2013_tray19_rgb.png',
 'ara2013_tray20_rgb.png',
 'ara2013_tray21_rgb.png',
 'ara2013_tray22_rgb.png',
 'ara2013_tray23_rgb.png',
 'ara2013_tray24_rgb.png',
 'ara2013_tray25_rgb.png',
 'ara2013_tray26_rgb.png',
 'ara2013_tray27_rgb.png',
 'ara2012_tray01_rgb.png',
 'ara2012_tray02_rgb.png',
 'ara2012_tray03_rgb.png',
 'ara2012_tray04_rgb.png',
 'ara2012_tray05_rgb.png',
 'ara2012_tray06_rgb.png',
 'ara2012_tray07_rgb.png',
 'ara2012_tray08_rgb.png',
 'ara2012_tray09_rgb.png',
 'ara2012_tray10_rgb.png']

In [97]:
os.listdir('/gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/plant_dataset/val/')

['ara2012_tray11_rgb.png',
 'ara2012_tray12_rgb.png',
 'ara2012_tray13_rgb.png',
 'ara2012_tray14_rgb.png',
 'ara2012_tray15_rgb.png',
 'ara2012_tray16_rgb.png']